# Project #1a: Text Classification with Logistic Regression

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Sample Data
data = {
    'text': [
        'The football match was exciting and full of action',
        'The government passed a new law today',
        'New advancements in artificial intelligence are remarkable',
        'The health benefits of yoga are numerous',
        'The basketball team won their championship game',
        'A major political scandal has rocked the nation',
        'Tech companies are investing in quantum computing',
        'A recent study shows the positive effects of meditation on mental health',
        'The tennis player won her third grand slam title',
        'The election results will be announced tomorrow'
    ],
    'category': ['Sports', 'Politics', 'Technology', 'Health', 'Sports', 'Politics', 'Technology', 'Health', 'Sports', 'Politics']
}
df = pd.DataFrame(data)

# Display the first few rows of the dataset
print(df.head())

# Split the data into features (X) and labels (y)
X = df['text']
y = df['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data:")
print(X_train)
print("Testing data:")
print(X_test)

# Initialize the CountVectorizer
bow_vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_bow = bow_vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_bow = bow_vectorizer.transform(X_test)

print("BoW Feature Names:")
print(bow_vectorizer.get_feature_names_out())
print("Training data (BoW):")
print(X_train_bow.toarray())

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("TF-IDF Feature Names:")
print(tfidf_vectorizer.get_feature_names_out())
print("Training data (TF-IDF):")
print(X_train_tfidf.toarray())

# Initialize the Logistic Regression model
lr_bow = LogisticRegression(max_iter=1000)

# Train the model
lr_bow.fit(X_train_bow, y_train)

# Make predictions
y_pred_bow = lr_bow.predict(X_test_bow)

# Evaluate the model
print("Accuracy (BoW):", accuracy_score(y_test, y_pred_bow))
print("Classification Report (BoW):")
print(classification_report(y_test, y_pred_bow))
print("Confusion Matrix (BoW):")
print(confusion_matrix(y_test, y_pred_bow))

# Initialize the Logistic Regression model
lr_tfidf = LogisticRegression(max_iter=1000)

# Train the model
lr_tfidf.fit(X_train_tfidf, y_train)

# Make predictions
y_pred_tfidf = lr_tfidf.predict(X_test_tfidf)

# Evaluate the model
print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):")
print(classification_report(y_test, y_pred_tfidf))
print("Confusion Matrix (TF-IDF):")
print(confusion_matrix(y_test, y_pred_tfidf))


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


                                                text    category
0  The football match was exciting and full of ac...      Sports
1              The government passed a new law today    Politics
2  New advancements in artificial intelligence ar...  Technology
3           The health benefits of yoga are numerous      Health
4    The basketball team won their championship game      Sports
Training data:
5      A major political scandal has rocked the nation
0    The football match was exciting and full of ac...
7    A recent study shows the positive effects of m...
2    New advancements in artificial intelligence ar...
9      The election results will be announced tomorrow
4      The basketball team won their championship game
3             The health benefits of yoga are numerous
6    Tech companies are investing in quantum computing
Name: text, dtype: object
Testing data:
8    The tennis player won her third grand slam title
1               The government passed a new law today
Name: t

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

# Project #1b: Text Classification by Fine-Tuning Pretrained LLM

In [2]:
# Import necessary libraries
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import numpy as np
import time
import datetime

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Sample Data 
data = {
    'text': [
        'The football match was exciting and full of action',
        'The government passed a new law today',
        'New advancements in artificial intelligence are remarkable',
        'The health benefits of yoga are numerous',
        'The basketball team won their championship game',
        'A major political scandal has rocked the nation',
        'Tech companies are investing in quantum computing',
        'A recent study shows the positive effects of meditation on mental health',
        'The tennis player won her third grand slam title',
        'The election results will be announced tomorrow'
    ],
    'category': ['Sports', 'Politics', 'Technology', 'Health', 'Sports', 'Politics', 'Technology', 'Health', 'Sports', 'Politics']
}
df = pd.DataFrame(data)

# Display the first few rows of the dataset to understand its structure
print(df.head())

# Encode the labels into integers
# LabelEncoder converts categorical labels (e.g., 'Sports', 'Politics') into numeric values
label_encoder = LabelEncoder()
df['category'] = label_encoder.fit_transform(df['category'])

# Split the data into features (X) and labels (y)
X = df['text']
y = df['category']

# Split the data into training and testing sets (80% train, 20% test)
# This helps in evaluating the performance of the model on unseen data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data:")
print(X_train)
print("Testing data:")
print(X_test)

# Define a custom dataset class to handle our data
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize the text and create attention masks
        # encode_plus helps in preparing the input in the required format for BERT
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Add [CLS] and [SEP] tokens
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',  # Pad to the maximum length
            return_attention_mask=True,  # Return attention mask to differentiate between padded and actual tokens
            return_tensors='pt',  # Return PyTorch tensors
            truncation=True,  # Truncate sequences longer than max_len
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Initialize the BERT tokenizer
# 'bert-base-uncased' refers to the pre-trained BERT model with uncased vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128  # Maximum length of input sequence

# Create dataset objects for training and testing data
train_dataset = NewsDataset(X_train.tolist(), y_train.tolist(), tokenizer, max_len)
test_dataset = NewsDataset(X_test.tolist(), y_test.tolist(), tokenizer, max_len)

# Create data loaders for training and testing data
# DataLoader helps in batching, shuffling, and loading the data in parallel
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Function to train the model for one epoch
def train_epoch(
    model, 
    data_loader, 
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    n_examples
):
    model = model.train()  # Set model to training mode
    losses = []
    correct_predictions = 0
    
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["label"].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        _, preds = torch.max(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, labels)
        
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        
        loss.backward()  # Compute gradients
        optimizer.step()  # Update model parameters
        scheduler.step()  # Update learning rate
        optimizer.zero_grad()  # Reset gradients
    
    # Calculate and return average accuracy and loss for this epoch
    return correct_predictions.double() / n_examples, np.mean(losses)

# Function to evaluate the model
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()  # Set model to evaluation mode
    losses = []
    correct_predictions = 0
    
    with torch.no_grad():  # Disable gradient computation
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["label"].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            _, preds = torch.max(outputs.logits, dim=1)
            loss = loss_fn(outputs.logits, labels)
            
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
    
    # Calculate and return average accuracy and loss for the evaluation
    return correct_predictions.double() / n_examples, np.mean(losses)

# Initialize the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels=len(label_encoder.classes_)  # Number of output labels
)
model = model.to(device)  # Move model to the appropriate device (GPU/CPU)

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_loader) * 3  # Total training steps (num_epochs * num_batches_per_epoch)

# Set up the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Define the loss function
loss_fn = torch.nn.CrossEntropyLoss().to(device)

# Dictionary to store training history
history = {'train_acc': [], 'train_loss': [], 'val_acc': [], 'val_loss': []}
best_accuracy = 0

# Train the model for 3 epochs
for epoch in range(3):
    print(f'Epoch {epoch + 1}/{3}')
    print('-' * 10)

    # Train the model for one epoch
    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(X_train)
    )
    
    print(f'Train loss {train_loss} accuracy {train_acc}')
    
    # Evaluate the model on the validation set
    val_acc, val_loss = eval_model(
        model,
        test_loader,
        loss_fn,
        device,
        len(X_test)
    )
    
    print(f'Val loss {val_loss} accuracy {val_acc}')
    
    # Store training and validation metrics
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    
    # Save the best model
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model_state.bin')

# Load the best model
model.load_state_dict(torch.load('best_model_state.bin'))

# Evaluate the model on the test data
test_acc, _ = eval_model(
    model,
    test_loader,
    loss_fn,
    device,
    len(X_test)
)

print(f'Test Accuracy: {test_acc}')

# Predictions and classification report
model = model.eval()  # Set model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad():
    for d in test_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["label"].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        _, preds = torch.max(outputs.logits, dim=1)
        
        predictions.extend(preds)
        true_labels.extend(labels)

# Stack predictions and true labels for evaluation
predictions = torch.stack(predictions).cpu()
true_labels = torch.stack(true_labels).cpu()

# Print classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_labels, predictions, labels=[0, 1, 2, 3], target_names=label_encoder.classes_, zero_division=1))
print("Confusion Matrix:")
print(confusion_matrix(true_labels, predictions, labels=[0, 1, 2, 3]))


                                                text    category
0  The football match was exciting and full of ac...      Sports
1              The government passed a new law today    Politics
2  New advancements in artificial intelligence ar...  Technology
3           The health benefits of yoga are numerous      Health
4    The basketball team won their championship game      Sports
Training data:
5      A major political scandal has rocked the nation
0    The football match was exciting and full of ac...
7    A recent study shows the positive effects of m...
2    New advancements in artificial intelligence ar...
9      The election results will be announced tomorrow
4      The basketball team won their championship game
3             The health benefits of yoga are numerous
6    Tech companies are investing in quantum computing
Name: text, dtype: object
Testing data:
8    The tennis player won her third grand slam title
1               The government passed a new law today
Name: t

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
----------
Train loss 1.3960583209991455 accuracy 0.125
Val loss 1.3168529272079468 accuracy 0.5
Epoch 2/3
----------
Train loss 1.415708065032959 accuracy 0.375
Val loss 1.277579426765442 accuracy 0.5
Epoch 3/3
----------
Train loss 1.2927600145339966 accuracy 0.625
Val loss 1.2550435066223145 accuracy 0.5
Test Accuracy: 0.5
Classification Report:
              precision    recall  f1-score   support

      Health       0.00      1.00      0.00         0
    Politics       1.00      0.00      0.00         1
      Sports       1.00      1.00      1.00         1
  Technology       1.00      1.00      1.00         0

   micro avg       0.50      0.50      0.50         2
   macro avg       0.75      0.75      0.50         2
weighted avg       1.00      0.50      0.50         2

Confusion Matrix:
[[0 0 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 0 0]]


# Project #2: Text Translation by Fine-Tuning Pretrained LLM

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
from datasets import load_metric
import numpy as np

# Sample bilingual data for translation
data = [
    {'en': 'Hello, how are you?', 'fr': 'Bonjour, comment ça va?'},
    {'en': 'What is your name?', 'fr': 'Comment tu t\'appelles?'},
    {'en': 'I am fine, thank you.', 'fr': 'Je vais bien, merci.'},
    {'en': 'Good morning', 'fr': 'Bonjour'},
    {'en': 'Good night', 'fr': 'Bonne nuit'},
    {'en': 'See you later', 'fr': 'À plus tard'},
    {'en': 'Thank you very much', 'fr': 'Merci beaucoup'},
    {'en': 'You are welcome', 'fr': 'Je vous en prie'},
    {'en': 'How much is this?', 'fr': 'Combien ça coûte?'},
    {'en': 'Where is the bathroom?', 'fr': 'Où sont les toilettes?'}
]

# Custom dataset class for translation data
class TranslationDataset(Dataset):
    def __init__(self, tokenizer, data, source_lang, target_lang, max_len):
        """
        Args:
            tokenizer: Pre-trained tokenizer for encoding the text
            data: List of dictionaries containing source and target language pairs
            source_lang: Source language key in the data dictionary
            target_lang: Target language key in the data dictionary
            max_len: Maximum sequence length for encoding
        """
        self.tokenizer = tokenizer
        self.data = data
        self.source_lang = source_lang
        self.target_lang = target_lang
        self.max_len = max_len

    def __len__(self):
        # Return the number of data points
        return len(self.data)

    def __getitem__(self, idx):
        # Retrieve source and target text for a given index
        source_text = self.data[idx][self.source_lang]
        target_text = self.data[idx][self.target_lang]

        # Tokenize and encode source text
        source = self.tokenizer.encode_plus(
            source_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Tokenize and encode target text
        target = self.tokenizer.encode_plus(
            target_text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Prepare labels for loss calculation, replacing padding tokens with -100
        labels = target['input_ids'].squeeze()
        labels[labels == 0] = -100

        return {
            'input_ids': source['input_ids'].squeeze(),
            'attention_mask': source['attention_mask'].squeeze(),
            'labels': labels
        }

# Initialize the tokenizer and model from pre-trained T5 base
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define dataset and parameters
max_len = 50  # Maximum sequence length
dataset = TranslationDataset(tokenizer, data, 'en', 'fr', max_len)

# Split data into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2)

# Define optimizer and scheduler for learning rate adjustment
optimizer = AdamW(model.parameters(), lr=5e-5, correct_bias=False)
total_steps = len(train_loader) * 3  # Total training steps for the scheduler

scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps
)

# Training function for one epoch
def train_epoch(model, data_loader, optimizer, device, scheduler):
    """
    Train the model for one epoch.
    
    Args:
        model: The model to train
        data_loader: DataLoader for the training data
        optimizer: Optimizer for the model
        device: Device (CPU or GPU) to run the training on
        scheduler: Learning rate scheduler
        
    Returns:
        Mean loss for the epoch
    """
    model.train()  # Set model to training mode
    losses = []

    for data in data_loader:
        # Move data to the specified device
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        losses.append(loss.item())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return np.mean(losses)

# Evaluation function
def eval_model(model, data_loader, device):
    """
    Evaluate the model on the validation data.
    
    Args:
        model: The model to evaluate
        data_loader: DataLoader for the validation data
        device: Device (CPU or GPU) to run the evaluation on
        
    Returns:
        Mean loss for the validation data
    """
    model.eval()  # Set model to evaluation mode
    losses = []

    with torch.no_grad():
        for data in data_loader:
            # Move data to the specified device
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            losses.append(loss.item())

    return np.mean(losses)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else CPU
model = model.to(device)

epochs = 10  # Number of training epochs
best_loss = float('inf')  # Initialize best loss to infinity

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
    print(f'Train loss: {train_loss}')

    val_loss = eval_model(model, val_loader, device)
    print(f'Validation loss: {val_loss}')

    # Save the best model based on validation loss
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'best_translation_model.bin')

# Load the best model for final evaluation
model.load_state_dict(torch.load('best_translation_model.bin'))

# Function to translate a single sentence
def translate_sentence(model, tokenizer, sentence, device, max_len=50):
    """
    Translate a given sentence using the trained model.
    
    Args:
        model: The trained model
        tokenizer: The tokenizer for encoding the input sentence
        sentence: The input sentence to translate
        device: Device (CPU or GPU) to run the translation on
        max_len: Maximum length for the output sentence
        
    Returns:
        The translated sentence
    """
    model.eval()
    input_ids = tokenizer.encode(sentence, return_tensors="pt").to(device)
    outputs = model.generate(input_ids, max_length=max_len, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the model with a few examples and calculate BLEU score
metric = load_metric("sacrebleu")

test_sentences = [
    {"en": "Hello, how are you?", "fr": "Bonjour, comment ça va?"},
    {"en": "What is your name?", "fr": "Comment tu t'appelles?"},
    {"en": "I am fine, thank you.", "fr": "Je vais bien, merci."}
]

references = []  # List to store reference translations
predictions = []  # List to store model predictions

for item in test_sentences:
    original_sentence = item["en"]
    reference_translation = item["fr"]
    translated_sentence = translate_sentence(model, tokenizer, original_sentence, device)

    # Print the original, translated, and reference sentences
    print(f'Original: {original_sentence}')
    print(f'Translated: {translated_sentence}')
    print(f'Reference: {reference_translation}\n')

    # Add the reference and predicted translations to their respective lists
    references.append([reference_translation])
    predictions.append(translated_sentence)

# Compute BLEU score for the translations
bleu_score = metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score['score']:.2f}")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
----------
Train loss: 3.7011754512786865
Validation loss: 2.441734552383423
Epoch 2/10
----------
Train loss: 2.5062824189662933
Validation loss: 2.2136828899383545
Epoch 3/10
----------
Train loss: 2.128346711397171
Validation loss: 2.1485533714294434
Epoch 4/10
----------
Train loss: 2.162066102027893
Validation loss: 2.1485533714294434
Epoch 5/10
----------
Train loss: 2.1558494567871094
Validation loss: 2.1485533714294434
Epoch 6/10
----------
Train loss: 2.1393941044807434
Validation loss: 2.1485533714294434
Epoch 7/10
----------
Train loss: 1.9143696129322052
Validation loss: 2.1485533714294434
Epoch 8/10
----------
Train loss: 1.9473647475242615
Validation loss: 2.1485533714294434
Epoch 9/10
----------
Train loss: 2.5014607310295105
Validation loss: 2.1485533714294434
Epoch 10/10
----------
Train loss: 2.190967470407486
Validation loss: 2.1485533714294434


/tmp/ipykernel_880290/639053091.py:216: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.8/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Original: Hello, how are you?
Translated: Hello, how are you?
Reference: Bonjour, comment ça va?

Original: What is your name?
Translated: Votre nom?
Reference: Comment tu t'appelles?

Original: I am fine, thank you.
Translated: Je suis bien, merci.
Reference: Je vais bien, merci.

BLEU score: 24.80


# Project #3a: Text Generation by Fine-Tuning Pretrained GPT-2

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Load and prepare the dataset
def load_data(file_path, tokenizer, block_size=256):
    dataset = load_dataset('text', data_files={'train': file_path})
    def tokenize_function(examples):
        return tokenizer(examples['text'], truncation=True, max_length=block_size, padding='max_length')
    tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
    return tokenized_datasets['train']

class TextDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.input_ids = tokenized_texts['input_ids']
        self.attention_mask = tokenized_texts['attention_mask']
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return torch.tensor(self.input_ids[idx]), torch.tensor(self.attention_mask[idx])

data_file = 'poems.txt'
tokenized_datasets = load_data(data_file, tokenizer)
dataset = TextDataset(tokenized_datasets)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

# Hyperparameters
num_epochs = 50
learning_rate = 3e-5

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for inputs, attention_masks in tqdm(dataloader):
        inputs, attention_masks = inputs.to(device), attention_masks.to(device)
        
        outputs = model(input_ids=inputs, attention_mask=attention_masks, labels=inputs)
        loss = outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader)}')

# Save the trained model
model.save_pretrained('./gpt2_model')
tokenizer.save_pretrained('./gpt2_tokenizer')

# Generate text with the trained GPT-2 model
def generate_text(model, tokenizer, prompt, max_length=50, temperature=1.0, top_k=20, top_p=0.95):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)
    with torch.no_grad():
        output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id, do_sample=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example prompts for text generation
prompts = [
    "But thou shrieking",
    "In the quiet",
    "On the sole",
    "Every fowl of",
    "In a mutual flame"
]

# Generate and display text based on the example prompts
for prompt in prompts:
    print(f"Prompt: {prompt}")
    generated_text = generate_text(model, tokenizer, prompt, max_length=50)
    print(f"Generated Text: {generated_text}\n")


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 14.52it/s]


Epoch 1/50, Loss: 1.8801447394348325


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 14.74it/s]


Epoch 2/50, Loss: 0.17307371823560624


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 14.74it/s]


...

Epoch 48/50, Loss: 0.13245863822244464


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 14.58it/s]


Epoch 49/50, Loss: 0.13368822172993705


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 14.57it/s]


Epoch 50/50, Loss: 0.13650958310990108
Prompt: But thou shrieking
Generated Text: But thou shrieking

Prompt: In the quiet
Generated Text: In the quiet

Prompt: On the sole
Generated Text: On the sole

Prompt: Every fowl of
Generated Text: Every fowl of

Prompt: In a mutual flame
Generated Text: In a mutual flame



# Project #3b: Text Generation with GPT-2 Tokenizer and Custom LSTM

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer
from datasets import load_dataset
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

# Load and prepare the dataset
def load_data(file_path, tokenizer, block_size=256):
    dataset = load_dataset('text', data_files={'train': file_path})
    def tokenize_function(examples):
        return tokenizer(examples['text'], truncation=True, max_length=block_size, padding='max_length')
    tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
    return tokenized_datasets['train']

class TextDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.input_ids = tokenized_texts['input_ids']
        self.attention_mask = tokenized_texts['attention_mask']
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return torch.tensor(self.input_ids[idx]), torch.tensor(self.attention_mask[idx])

data_file = 'poems.txt'
tokenized_datasets = load_data(data_file, tokenizer)
dataset = TextDataset(tokenized_datasets)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden):
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        logits = self.fc(lstm_out)
        return logits, hidden

# Hyperparameters
vocab_size = len(tokenizer)
embedding_dim = 256
hidden_dim = 512
num_layers = 5
num_epochs = 50
learning_rate = 3e-5

# Initialize the model, loss function, and optimizer
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    hidden = (torch.zeros(num_layers, 4, hidden_dim).to(device),
              torch.zeros(num_layers, 4, hidden_dim).to(device))  # Initialize hidden state and cell state
    for inputs, _ in tqdm(dataloader):
        inputs = inputs.to(device)
        
        optimizer.zero_grad()
        
        outputs, hidden = model(inputs, hidden)
        hidden = (hidden[0].detach(), hidden[1].detach())  # Detach hidden state and cell state for truncated BPTT
        loss = loss_fn(outputs.view(-1, vocab_size), inputs.view(-1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader)}')

# Save the trained model
torch.save(model.state_dict(), './lstm_model.pth')

# Generate text with the trained LSTM model
def generate_text(model, tokenizer, prompt, max_length=50, temperature=1.0):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    input_ids = input_ids[:, :-1]  # Remove the EOS token to prevent it from being considered a part of the prompt
    hidden = (torch.zeros(num_layers, 1, hidden_dim).to(device),
              torch.zeros(num_layers, 1, hidden_dim).to(device))  # Initialize hidden state and cell state
    
    generated = input_ids
    for _ in range(max_length):
        outputs, hidden = model(generated[:, -1].unsqueeze(0), hidden)
        next_token_logits = outputs[0, -1, :] / temperature
        next_token = torch.multinomial(torch.softmax(next_token_logits, dim=-1), num_samples=1)
        generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
        if next_token.item() == tokenizer.eos_token_id:
            break
    
    return tokenizer.decode(generated[0], skip_special_tokens=True)

# Example prompts for text generation
prompts = [
    "But thou shrieking",
    "In the quiet",
    "On the sole",
    "Every fowl of",
    "In a mutual flame"
]

# Generate and display text based on the example prompts
for prompt in prompts:
    print(f"Prompt: {prompt}")
    generated_text = generate_text(model, tokenizer, prompt, max_length=50)
    print(f"Generated Text: {generated_text}\n")


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 23.91it/s]


Epoch 1/50, Loss: 10.776877584911528


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 25.02it/s]


Epoch 2/50, Loss: 9.596878732953753


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 24.83it/s]


...

Epoch 48/50, Loss: 0.2453618578258015


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 25.00it/s]


Epoch 49/50, Loss: 0.2464179134085065


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 24.79it/s]


Epoch 50/50, Loss: 0.24540535325095766
Prompt: But thou shrieking
Generated Text: But thou shriPennMilitary cable inhathon essayector Cob Sinn did

Prompt: In the quiet
Generated Text: In the hers mimic robotic RG turned Threat

Prompt: On the sole
Generated Text: On the pressures snowy much indexes Ident Michel

Prompt: Every fowl of
Generated Text: Every fowlWay Holo Solutionnone restart guy dietrowerctl Wake writes effort

Prompt: In a mutual flame
Generated Text: In a mutualsubmit lakeXP Ramsay hemorCD Boise skepticism



# Project #3c: Text Generation with Custom Tokenizer and Custom LSTM

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import re
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom tokenizer from scratch
class CustomTokenizer:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = 0

    def build_vocab(self, texts):
        counter = Counter()
        for text in texts:
            tokens = self.tokenize(text)
            counter.update(tokens)
        
        self.vocab_size = len(counter) + 2  # Adding 2 for PAD and EOS tokens
        self.word2idx = {word: idx + 2 for idx, (word, _) in enumerate(counter.most_common())}
        self.word2idx['<PAD>'] = 0
        self.word2idx['<EOS>'] = 1
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}
    
    def tokenize(self, text):
        text = re.sub(r'[\W_]+', ' ', text).lower().strip()
        return text.split()

    def encode(self, text, max_length=256):
        tokens = self.tokenize(text)
        token_ids = [self.word2idx.get(token, self.word2idx['<PAD>']) for token in tokens]
        token_ids = token_ids[:max_length-1] + [self.word2idx['<EOS>']]
        padding = [self.word2idx['<PAD>']] * (max_length - len(token_ids))
        return token_ids + padding

    def decode(self, token_ids):
        tokens = [self.idx2word[token_id] for token_id in token_ids if token_id not in {self.word2idx['<PAD>'], self.word2idx['<EOS>']}]
        return ' '.join(tokens)

# Load and prepare the dataset
def load_data(file_path, tokenizer, block_size=256):
    with open(file_path, 'r') as f:
        texts = f.readlines()
    
    tokenizer.build_vocab(texts)
    tokenized_texts = [tokenizer.encode(text, max_length=block_size) for text in texts]
    return tokenized_texts

class TextDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.input_ids = tokenized_texts
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return torch.tensor(self.input_ids[idx]), torch.tensor(self.input_ids[idx])

data_file = 'poems.txt'
tokenizer = CustomTokenizer()
tokenized_texts = load_data(data_file, tokenizer)
dataset = TextDataset(tokenized_texts)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden):
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        logits = self.fc(lstm_out)
        return logits, hidden

# Hyperparameters
vocab_size = tokenizer.vocab_size
embedding_dim = 256
hidden_dim = 512
num_layers = 5
num_epochs = 50
learning_rate = 3e-5

# Initialize the model, loss function, and optimizer
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.word2idx['<PAD>'])
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    hidden = (torch.zeros(num_layers, 4, hidden_dim).to(device),
              torch.zeros(num_layers, 4, hidden_dim).to(device))  # Initialize hidden state and cell state
    for inputs, _ in tqdm(dataloader):
        inputs = inputs.to(device)
        
        optimizer.zero_grad()
        
        outputs, hidden = model(inputs, hidden)
        hidden = (hidden[0].detach(), hidden[1].detach())  # Detach hidden state and cell state for truncated BPTT
        loss = loss_fn(outputs.view(-1, vocab_size), inputs.view(-1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader)}')

# Save the trained model
torch.save(model.state_dict(), './lstm_model.pth')

# Generate text with the trained LSTM model
def generate_text(model, tokenizer, prompt, max_length=50, temperature=1.0):
    model.eval()
    input_ids = torch.tensor(tokenizer.encode(prompt)[:-1]).unsqueeze(0).to(device)  # Remove the EOS token to prevent it from being considered a part of the prompt
    hidden = (torch.zeros(num_layers, 1, hidden_dim).to(device),
              torch.zeros(num_layers, 1, hidden_dim).to(device))  # Initialize hidden state and cell state
    
    generated = input_ids
    for _ in range(max_length):
        outputs, hidden = model(generated[:, -1].unsqueeze(0), hidden)
        next_token_logits = outputs[0, -1, :] / temperature
        next_token = torch.multinomial(torch.softmax(next_token_logits, dim=-1), num_samples=1)
        generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
        if next_token.item() == tokenizer.word2idx['<EOS>']:
            break
    
    return tokenizer.decode(generated[0].tolist())

# Example prompts for text generation
prompts = [
    "But thou shrieking",
    "In the quiet",
    "On the sole",
    "Every fowl of",
    "In a mutual flame"
]

# Generate and display text based on the example prompts
for prompt in prompts:
    print(f"Prompt: {prompt}")
    generated_text = generate_text(model, tokenizer, prompt, max_length=50)
    print(f"Generated Text: {generated_text}\n")


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 33.54it/s]


Epoch 1/50, Loss: 5.397375447409494


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 34.01it/s]


Epoch 2/50, Loss: 5.369072005862281


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 33.60it/s]


...

Epoch 48/50, Loss: 4.465279125031971


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 32.79it/s]


Epoch 49/50, Loss: 4.60325152533395


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 33.27it/s]


Epoch 50/50, Loss: 4.560289019630069
Prompt: But thou shrieking
Generated Text: but thou shrieking lack repair mine divining simple arabian right how

Prompt: In the quiet
Generated Text: in the is she this of

Prompt: On the sole
Generated Text: on the sole distincts loudest are co rarity itself reason lov mutual twixt

Prompt: Every fowl of
Generated Text: every fowl of rest reason called eagle queen

Prompt: In a mutual flame
Generated Text: in a mutual flame whose precurrer

